In [1]:
import gzip
import glob
import json
import pandas as pd
import numpy as np

In [2]:
path2 = '/Users/Meghna/Desktop/labels_predictions_twitter_test_images.csv'

In [3]:
df2 = pd.read_csv(path2)

In [4]:
df2.head()

,label,prediction,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,5.770470e+07,0.106601,NaN,NaN,NaN,NaN,NaN
1,1.492350e+07,0.009944,NaN,NaN,NaN,NaN,NaN
2,7.033892e+17,0.144770,NaN,NaN,NaN,NaN,NaN
3,6.398567e+07,0.008392,NaN,NaN,NaN,NaN,NaN
4,1.071462e+18,0.613577,NaN,NaN,NaN,NaN,NaN


In [5]:
data_df = pd.DataFrame()

In [6]:
data_df['label'] = df2['label']
data_df['prediction'] = df2['prediction']

In [7]:
data_df.head()

,label,prediction
0,5.770470e+07,0.106601
1,1.492350e+07,0.009944
2,7.033892e+17,0.144770
3,6.398567e+07,0.008392
4,1.071462e+18,0.613577


In [8]:
probas = np.array(data_df['prediction'].to_list())
labels = (probas > 0.5).astype(np.int)
print(labels)

[0 0 0 ... 0 1 0]


In [9]:
sum(labels)

16893

In [10]:
race_dict = {}
races = list(labels)
user_ids = data_df['label'].to_list()
for i in range(len(user_ids)):
    race_dict[int(user_ids[i])] = races[i]
#race_dict = df.set_index('user_id').T.to_dict('str')
twitter_race_tweets = {}

In [11]:
#print(race_dict)

In [12]:
path = '/Users/Meghna/Desktop/Twitter_Covid_Data/*.json.gz'
files = glob.glob(path)

In [13]:
df_race_tweets = pd.DataFrame()

In [14]:
for i in range(len(files)):
    try:
        with gzip.open(files[i], 'rb') as f:
            for line in f:
                data = json.loads(line)
                try:
                    uid = data['user']['id']
                except:
                    continue
                if uid in race_dict.keys():
                    twitter_race_tweets['user_id'] = uid
                    twitter_race_tweets['race'] = race_dict[uid]
                    if data['truncated'] == True:
                        twitter_race_tweets['tweet'] = data['extended_tweet']['full_text']
                    else:
                        twitter_race_tweets['tweet'] = data['text']
                    #json.dump(twitter_race_tweets,n)
                    #n.write('\n')
                    df_race_tweets = df_race_tweets.append(twitter_race_tweets, ignore_index=True)
                    df_race_tweets['race'] = df_race_tweets['race'].astype('int')
                    df_race_tweets['user_id'] = df_race_tweets['user_id'].astype('int')
                    twitter_race_tweets.clear()
    except Exception as e:
        print(e)
        print("Faulty file ", files[i]) #some files are faulty in the retrieved dataset and do not open



Error -3 while decompressing data: invalid block type
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1607015151542.json.gz
CRC check failed 0x8088b1f != 0xfb1bf383
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1607032511977.json.gz
Error -3 while decompressing data: invalid block type
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1607028452380.json.gz
CRC check failed 0x8088b1f != 0xcedc3b20
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1606757032233.json.gz
Error -3 while decompressing data: invalid block type
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1607021468820.json.gz
CRC check failed 0x8088b1f != 0x91876296
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1606770871554.json.gz
Error -3 while decompressing data: invalid block type
Faulty file  /Users/Meghna/Desktop/Twitter_Covid_Data/covid19-1606777826793.json.gz
Error -3 while decompressing data: invalid block type
Faulty file  /Users

In [15]:
df_race_tweets

,race,tweet,user_id
0,0,And on the first day of the Colorado legislatu...,195850342
1,1,Everyone needs to read this. Dr. Nichols and h...,15584648
2,1,RT @dannygonzalez: streamys did something righ...,4908500380
3,0,RT @marginalmarisa: everyone who’s like “nobod...,1071998764876029952
4,0,“He would make outrageous claims like he inven...,50961846
...,...,...,...
146193,0,RT @deeksj: Mass testing in Liverpool MISSED ~...,262178588
146194,0,God Bless you Mike! 😔,1011686593
146195,1,"RT @103blinddog: Prince Harry says, Pandemic i...",1395038046
146196,0,@gwenmoritz Wear a mask. Social distance. Wash...,5409072


In [16]:
df_race_tweets.to_csv('/Users/Meghna/Desktop/twitter_race_tweets.csv')